# Unzip the dataset

In [7]:
import zipfile
import os

def unzip_file(zip_file_path, extract_to_path):
  """
  Unzips a zip file to a specified directory.

  Args:
      zip_file_path: The path to the zip file.
      extract_to_path: The path to the directory where contents should be extracted.
  """
  try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extract_to_path)
    print(f"Successfully unzipped '{zip_file_path}' to '{extract_to_path}'")
  except FileNotFoundError:
    print(f"Error: Zip file not found at '{zip_file_path}'")
  except zipfile.BadZipFile:
    print(f"Error: Invalid zip file: '{zip_file_path}'")
  except Exception as e:
    print(f"An error occurred: {e}")

# Example usage (replace with your file paths):
zip_file_path = 'lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6.zip'  # Path to the zip file you want to unzip
extract_to_path = 'lmd-full_and_reddit_MIDI_dataset/'  # Path to the directory to extract to

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

unzip_file(zip_file_path, extract_to_path)

Successfully unzipped 'lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6.zip' to 'lmd-full_and_reddit_MIDI_dataset/'


# Explore a npy file

In [33]:
import numpy as np

def inspect_npy_file(file_path):
    """
    Loads and inspects the contents of a .npy file.

    Args:
        file_path: The path to the .npy file.

    Returns:
        None. Prints information about the array stored in the file.
    """
    try:
        # Load the .npy file
        data = np.load(file_path, allow_pickle=True)

        # Print information about the array
        print(f"File: {file_path}")
        print(f"Data type: {data.dtype}")
        print(f"Shape: {data.shape}")

        if data.dtype == 'object':
            print("The array contains objects.")
            if data.ndim == 0:
                # Handle 0-dimensional object arrays (scalar objects)
                print(f"  Object Type: {type(data.item())}")
                if isinstance(data.item(), dict):
                    print("  Keys:", data.item().keys())
                else:
                    print(f"  Value: {data.item()}")
            else:
                # Handle multi-dimensional object arrays
                for i in range(data.shape[0]):
                    for j in range(data.shape[1]):
                        item = data[i, j]
                        print(f"  Item at index ({i}, {j}):")
                        print(f"    Object Type: {type(item)}")
                        if isinstance(item, dict):
                            print("    Keys:", item.keys())
                        else:
                            print(f"    Value: {item}")

        else:
            # Handle numerical arrays
            print("First few elements (up to 10):")
            if data.ndim == 1:
                print(data[:10])
            elif data.ndim == 2:
                print(data[:min(data.shape[0], 10), :min(data.shape[1], 10)])
            else:
                print(data.flatten()[:10])

            # If it's a small array, print the entire array
            if data.size < 20:
                print("Full array:")
                print(data)

        # Handle metadata (if present)
        if hasattr(data, 'metadata') and data.metadata is not None:
            print("\nMetadata:")
            for key, value in data.metadata.items():
                print(f"  {key}: {value}")

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
file_path = "lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6/kate_nash-do_wah_doo.npy"


inspect_npy_file(file_path)

File: lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6/kate_nash-do_wah_doo.npy
Data type: object
Shape: (1, 4)
The array contains objects.
  Item at index (0, 0):
    Object Type: <class 'list'>
    Value: [[[[0.0, 0.17666666666666675, 174.61411571650194, 97.0]], [[0.19166666666666643, 0.18500000000000005, 174.61411571650194, 82.0]]], [[[0.19500000000000028, 0.18999999999999995, 174.61411571650194, 90.0]]], [[[0.21333333333333337, 0.1766666666666663, 220.0, 82.0]]], [[[0.19416666666666638, 0.1825000000000001, 220.0, 92.0]]], [[[6.800000000000001, 0.18499999999999872, 440.0, 108.0], [0.20250000000000057, 0.08249999999999957, 440.0, 96.0], [0.20083333333333186, 0.18166666666666842, 391.99543598174927, 102.0], [0.1999999999999993, 0.081666666666667, 391.99543598174927, 93.0]], [[0.20083333333333542, 0.3808333333333316, 349.2282314330039, 105.0]], [[0.39833333333333343, 0.18249999999999744, 329.6275569128699, 99.0]], [[0.7991666666666646, 0.5833333333333357, 329.6275569128699, 105.0]

# Make midi files out of the npy files

In [ ]:
import numpy as np
import pretty_midi
import os

def npy_to_midi_discrete(npy_file, midi_file, tempo=120):
    """
    Converts a .npy file with encoded musical data (discrete attributes only)
    into a MIDI file.

    Args:
        npy_file: Path to the input .npy file.
        midi_file: Path to the output MIDI file (.mid).
        tempo: Tempo of the MIDI file in beats per minute (BPM).
    """

    # Load the .npy file
    data = np.load(npy_file, allow_pickle=True)

    # Create a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)

    # Create an instrument instance for a piano (instrument number 0)
    instrument = pretty_midi.Instrument(program=0)

    # Assuming data[0, 1] is discrete
    discrete_data = data[0, 1]

    # Keep track of the current time in the MIDI file
    current_time = 0

    for sentence_index, sentence in enumerate(discrete_data):
        for word_index, word in enumerate(sentence):
            for note_index, note in enumerate(word):
                pitch = int(note[0])
                duration_staves = note[1]
                rest_duration_staves = note[2]

                # Calculate note duration in seconds based on tempo and staves
                note_duration_seconds = (60 / tempo) * duration_staves

                # Calculate rest duration in seconds
                rest_duration_seconds = (60 / tempo) * rest_duration_staves

                # Advance current time by the rest duration
                current_time += rest_duration_seconds

                # Create a PrettyMIDI Note instance
                pm_note = pretty_midi.Note(
                    velocity=100,  # Default velocity
                    pitch=pitch,
                    start=current_time,
                    end=current_time + note_duration_seconds
                )

                # Add the note to the instrument
                instrument.notes.append(pm_note)

                # Advance current time by the note duration
                current_time += note_duration_seconds

    # Add the instrument to the MIDI object
    midi.instruments.append(instrument)

    # Write the MIDI data to a file
    midi.write(midi_file)
    print(f"MIDI file saved to: {midi_file}")

def convert_folder_to_midi(npy_folder, midi_folder, tempo=120):
    """
    Converts all .npy files in a folder to MIDI files using discrete data.

    Args:
        npy_folder: Path to the folder containing .npy files.
        midi_folder: Path to the folder where MIDI files will be saved.
        tempo: Tempo for the MIDI files in BPM.
    """

    # Create the MIDI output folder if it doesn't exist
    if not os.path.exists(midi_folder):
        os.makedirs(midi_folder)

    # Iterate through all files in the .npy folder
    for filename in os.listdir(npy_folder):
        if filename.endswith(".npy"):
            npy_path = os.path.join(npy_folder, filename)
            midi_filename = os.path.splitext(filename)[0] + ".mid"
            midi_path = os.path.join(midi_folder, midi_filename)

            try:
                npy_to_midi_discrete(npy_path, midi_path, tempo=tempo)
            except Exception as e:
                print(f"Error converting {filename}: {e}")

# Example usage:
npy_folder = "lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6"  # Replace with your .npy folder
midi_folder = "lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6_MIDI"  # Replace with your desired output folder
tempo = 120  # Set the desired tempo

convert_folder_to_midi(npy_folder, midi_folder, tempo)

# Make a pandas dataframe containing the lyrics and the path to the midi files

In [35]:
import numpy as np
import pandas as pd
import os

def extract_lyrics(npy_file):
    """
    Extracts lyrics from a .npy file.

    Args:
        npy_file: Path to the input .npy file.

    Returns:
        A string containing the extracted lyrics.
    """
    data = np.load(npy_file, allow_pickle=True)

    # Assuming lyrics data is in data[0, 3]
    word_list = data[0, 3]

    lyrics = ""
    for phrase in word_list:
        line = ""
        for part in phrase:
            line += "".join(part) + " "
        lyrics += line.strip() + "\n"
    return lyrics

def create_lyrics_midi_dataframe(npy_folder, midi_folder):
    """
    Creates a Pandas DataFrame with lyrics and corresponding MIDI file paths.

    Args:
        npy_folder: Path to the folder containing .npy files.
        midi_folder: Path to the folder containing the corresponding MIDI files.

    Returns:
        A Pandas DataFrame with 'lyrics' and 'midi_path' columns.
    """
    data = []
    for filename in os.listdir(npy_folder):
        if filename.endswith(".npy"):
            npy_path = os.path.join(npy_folder, filename)
            midi_filename = os.path.splitext(filename)[0] + ".mid"
            midi_path = os.path.join(midi_folder, midi_filename)

            # Check if the corresponding MIDI file exists
            if os.path.exists(midi_path):
                try:
                    lyrics = extract_lyrics(npy_path)
                    if lyrics.strip():  # Check if lyrics are not empty after removing whitespace
                        data.append({'lyrics': lyrics, 'midi_path': midi_path})
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
            else:
                print(f"MIDI file not found for {filename}")

    return pd.DataFrame(data)

# Example usage:
npy_folder = "lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6"  # Replace with your .npy folder
midi_folder = "lmd-full_and_reddit_MIDI_dataset/sentenceWord_level_6_MIDI"  # Replace with your MIDI folder

df = create_lyrics_midi_dataframe(npy_folder, midi_folder)
df.head()

# Save to CSV (optional)
# df.to_csv("lyrics_midi_data.csv", index=False)

,lyrics,midi_path
0,In sleep he sang to me\nin dreams he came\ntha...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
1,I have plans and schemes\nAnd I have hopes and...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
2,I get up and nothing gets me You got\nit tough...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
3,Man a hot like seven inches\nfrom the midday I...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
4,We come from the land of the ice and snow\nfro...,lmd-full_and_reddit_MIDI_dataset/sentenceWord_...
